<a href="https://colab.research.google.com/github/hugomedsa/separador_de_melodia/blob/main/VocalRemover_Spleeter_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub spleeter imageio-ffmpeg pygame PyTube -q

###CÓDIGO ESTRUTURADO

In [11]:
from IPython.display import HTML, Audio
from google.colab import files
from time import sleep
from pytube import YouTube
import os, subprocess, spleeter,pygame,io,torchaudio, ffmpeg
from pydub import AudioSegment
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read

def yt():
  link=input('Insira o link: ')
  video = YouTube(link).streams.filter(only_audio=False,bitrate="48kbps").first() #
  out_file = video.download()
  n = 0
  base, ext = os.path.splitext(out_file)

  inic_fim =  '00:'+'00:05'              + ':' +           '00:' +'01:30'
  sec = [int(c) for c in inic_fim.split(':')]
  t1 = sec[0]*3600 + sec[1]*60 + sec[2]
  t2 = sec[3]*3600 + sec[4]*60 + sec[5]
  
  ffmpeg_extract_subclip(out_file,t1, t2, targetname=f'{base} ({n}).mp4')
  old_file = f'{base} ({n}).mp4'
  base2, ext = os.path.splitext(old_file)
  new_file = base2 + '.mp3'
  n +=1
  os.rename(old_file, new_file)
  b = new_file[9:]
  return b

def up():
  b = list(files.upload().keys())[0] #FAZER UPLOAD DO AUDIO DO PC
  return b

def rec():
  AUDIO_HTML = """
  <script>
  var my_div = document.createElement("DIV");
  var my_p = document.createElement("P");
  var my_btn = document.createElement("BUTTON");
  var t = document.createTextNode("Press to start recording");

  my_btn.appendChild(t);
  //my_p.appendChild(my_btn);
  my_div.appendChild(my_btn);
  document.body.appendChild(my_div);

  var base64data = 0;
  var reader;
  var recorder, gumStream;
  var recordButton = my_btn;

  var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
      //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
      mimeType : 'audio/webm;codecs=opus'
      //mimeType : 'audio/webm;codecs=pcm'
    };            
    //recorder = new MediaRecorder(stream, options);
    recorder = new MediaRecorder(stream);
    recorder.ondataavailable = function(e) {            
      var url = URL.createObjectURL(e.data);
      var preview = document.createElement('audio');
      preview.controls = true;
      preview.src = url;
      document.body.appendChild(preview);

      reader = new FileReader();
      reader.readAsDataURL(e.data); 
      reader.onloadend = function() {
        base64data = reader.result;
        //console.log("Inside FileReader:" + base64data);
      }
    };
    recorder.start();
    };

  recordButton.innerText = "Recording... press to stop";

  navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


  function toggleRecording() {
    if (recorder && recorder.state == "recording") {
        recorder.stop();
        gumStream.getAudioTracks()[0].stop();
        recordButton.innerText = "Saving the recording... pls wait!"
    }
  }

  // https://stackoverflow.com/a/951057
  function sleep(ms) {
    return new Promise(resolve => setTimeout(resolve, ms));
  }

  var data = new Promise(resolve=>{
  //recordButton.addEventListener("click", toggleRecording);
  recordButton.onclick = ()=>{
  toggleRecording()

  sleep(2000).then(() => {
    // wait 2000ms for the data to be available...
    // ideally this should use something like await...
    //console.log("Inside data:" + base64data)
    resolve(base64data.toString())

  });

  }
  });
        
  </script>
  """
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open('audio.mp3','wb') as f:
    f.write(binary)
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]
  sr, audio = wav_read(io.BytesIO(riff))
  canto = io.BytesIO(riff)
  return 'audio.mp3'

###EXECUÇÃO

In [ ]:
b = yt() # yt(Extrair áudio do YouTube); up(Fazer upload do arquivo); rec(Usar gravação de voz ambiente)
os.rename(b, 'a.mp3')
!spleeter separate -p spleeter:2stems -o output/ a.mp3 -b 48k -c mp3  #spleeter versao 1.4.0 INCLUINDO ACOMPLISHMENT -b 64k
audio_final = 'output/a/'+b+'.mp3'
os.rename('output/a/vocals.mp3',audio_final)
sound = AudioSegment.from_mp3(audio_final)
twenty_sec_silence = AudioSegment.silent(duration=2000) #20 segundos no final do audio
sound_with_silence = sound + twenty_sec_silence
sound_with_silence.export('audio_silence.mp3')
files.download('audio_silence.mp3')
os.rename('a.mp3',b)